In [1]:
# Load environment variables from .env file
%load_ext dotenv
%dotenv

In [5]:
from langgraph.graph import add_messages, StateGraph
from typing_extensions import TypedDict
from typing import Annotated
from langgraph.checkpoint.sqlite import SqliteSaver

class State(TypedDict):
    # Messages have the type "list". The `add_messages` function
    # in the annotation defines how this state key should be updated
    # (in this case, it appends messages to the list, rather than overwriting them)
    messages: Annotated[list, add_messages]
    question: str
    
graph_builder = StateGraph(State)

In [ ]:
vector_store = CouchbaseVectorStore(
    cluster=cluster,
    bucket_name=BUCKET_NAME,
    scope_name=SCOPE_NAME,
    collection_name=COLLECTION_NAME,
    embedding=embedding,
    index_name=document["search_index_name"],
)

In [6]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo")

def chatbot(state: State):
    context = " ".join(state['messages'])  # Use all messages as context
    response = llm.generate(context)
    # Add the response to the messages
    state['messages'].append(f"Chatbot response: {response}")
    return state

def search_couchbase(state: State):
    query = state['question']
    retriever = vector_store.as_retriever()
    results = retriever.invoke(query)
    # Add the results to the messages
    state['messages'].append(f"Search results: {results}")
    return state

graph_builder.add_node("vector_search", search_couchbase)
graph_builder.add_node("chatbot", chatbot)
graph_builder.set_entry_point("vector_search")
graph_builder.add_edge("vector_search", "chatbot")

memory = SqliteSaver.from_conn_string(":memory:")
graph = graph_builder.compile(checkpointer=memory)

In [ ]:
while True:
    user_input = input("User: ")
    if user_input.lower() in ["quit", "exit", "q"]:
        print("Goodbye!")
        break
    for event in graph.stream({"messages": ("user", user_input)}):
        for value in event.values():
            print("Assistant:", value["messages"][-1].content)